# Proyecto - Datos

## Tratamiento de datos

In [85]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler

In [2]:
#Tratamiento individual de cada csv
for tiempo in range(1, 8+1):
    datos_df = pd.read_csv('./Datos/MiBici/Originales/datos_abiertos_2024_0{}.csv'.format(tiempo), index_col=0)
    
    #Tratamiento de los nombres
    datos_df=datos_df.reset_index()
    datos_df.rename(columns = {
                    'Usuario_Id': 'Usuario',
                    'Año_de_nacimiento': 'Año_nacimiento'
                    }, inplace = True)
    datos_df.columns = [col.lower() for col in datos_df]

    #Columnas en formato de fecha
    datos_df['inicio_del_viaje'] = pd.to_datetime(datos_df['inicio_del_viaje'])
    datos_df['fin_del_viaje'] = pd.to_datetime(datos_df['fin_del_viaje'])

    #Eliminación de datos de prueba
    datos_df = datos_df[~(
            (datos_df['inicio_del_viaje'].dt.minute == 0) & (datos_df['inicio_del_viaje'].dt.second == 0) &
            (datos_df['fin_del_viaje'].dt.minute == 0) & (datos_df['fin_del_viaje'].dt.second == 0))]

    #Columna tiempo total
    datos_df['tiempo_total'] = datos_df['fin_del_viaje'] - datos_df['inicio_del_viaje']
    
    #Columna día de la semana
    datos_df['dia_semana'] = datos_df['inicio_del_viaje'].dt.day_name()
    
    #Imputación de valores nulos
    imp = datos_df.groupby('dia_semana').agg({
                'año_nacimiento': 'mean',
                'genero': lambda x: 
                x.mode()[0] if not x.mode().empty else 'Desconocido'})
    datos_df['genero'] = datos_df['genero'].fillna(
                datos_df['dia_semana'].map(lambda dia: imp.loc[dia, 'genero']))
    datos_df['año_nacimiento'] = datos_df['año_nacimiento'].fillna(
                datos_df['dia_semana'].map(lambda dia: imp.loc[dia, 'año_nacimiento']))

    #Años en enteros
    datos_df['año_nacimiento']=datos_df['año_nacimiento'].astype(int)

    #Eliminación de usuarios con años de nacimiento menores a 1940
    datos_df=datos_df[datos_df['año_nacimiento'] >= 1940]
    
    #Columna edad
    datos_df['edad'] = 2024 - datos_df['año_nacimiento']
    
    #Eliminar viajes de 0 segundos
    datos_df = datos_df[datos_df['tiempo_total'].dt.total_seconds() > 0]
    
    #Eliminar viajes a la misma estación de menos de 3 minutos
    datos_df = datos_df[~((datos_df['origen_id'] == datos_df['destino_id']) & (datos_df['tiempo_total'].dt.total_seconds() < 180))]
    
    #Agregar columnas de distancias
    dis_df = pd.read_csv('./Datos/MiBici/Distancias_GoogleMaps_2024.csv')
    datos_df = datos_df.merge(
            dis_df[['origen_id', 'destino_id', 'googlemaps_dis', 'geodesic_dis']],
            on=['origen_id', 'destino_id'],
            how='left')
    datos_df = datos_df.set_index('viaje_id')

    #Distancias para estaciones repetidas
    def distancia(tiempo_total, vel_prom, percentil_alto):
        if (tiempo_total.total_seconds() > percentil_alto):
            return 20000 / (1 + np.exp(-0.0003 * (tiempo - 6000)))
        return (tiempo_total.total_seconds()/3600 * vel_prom * 1000)
    
    p90 = datos_df['tiempo_total'].dt.total_seconds().quantile(0.9)
    
    vel_prom_maps = (datos_df.loc[~np.isinf(datos_df['googlemaps_dis']),'googlemaps_dis'].mean()/(1000*datos_df['tiempo_total'].dt.total_seconds()/3600).mean())
    vel_prom_geo = (datos_df.loc[~np.isinf(datos_df['geodesic_dis']),'geodesic_dis'].mean()/(1000*datos_df['tiempo_total'].dt.total_seconds()/3600).mean())
    datos_df.loc[datos_df['origen_id'] == datos_df['destino_id'], 'googlemaps_dis'] = datos_df.apply(lambda row: distancia(row['tiempo_total'], vel_prom_maps, p90), axis=1)
    datos_df.loc[datos_df['origen_id'] == datos_df['destino_id'], 'geodesic_dis'] = datos_df.apply(lambda row: distancia(row['tiempo_total'], vel_prom_geo, p90), axis=1)
    
    #Columnas de velocidad
    datos_df['vel_prom_maps'] = (datos_df['googlemaps_dis'] / (1000*datos_df['tiempo_total'].dt.total_seconds()/3600)).round(2)
    datos_df['vel_prom_geo'] = (datos_df['geodesic_dis'] / (1000*datos_df['tiempo_total'].dt.total_seconds()/3600)).round(2)
    
    #Eliminar datos con velocidades mayores a 50km/h
    datos_df = datos_df[datos_df['vel_prom_geo'] <= 50]
    
    #Inclusión de tarifas
    def tarifa(tiempo_total, dia_semana):
        t=tiempo_total.total_seconds()/60
        n = 45 if(dia_semana=='Sunday') else 30
        if(t<=n):
            return 0
        elif(t<=2*n):
            return 28
        else:
            return min(39*((t-2*n)//n+1)+28, 1052)
    
    datos_df['tarifa'] = datos_df.apply(lambda row: tarifa(row['tiempo_total'], row['dia_semana']), axis=1)
    
    #Convertir a enteros
    datos_df['googlemaps_dis'] = datos_df['googlemaps_dis'].astype(int)
    datos_df['geodesic_dis'] = datos_df['geodesic_dis'].astype(int)
    datos_df['tarifa'] = datos_df['tarifa'].astype(int)
    
    #Guardar cada archivo
    datos_df.to_csv('./Datos/MiBici/datos_tratados_2024_0{}.csv'.format(tiempo))

---
## Manejo de los datos

In [3]:
#Leer y concatenar dataframes
year2024=[]

for tiempo in range(1, 8+1):
    year2024.append(pd.read_csv('./Datos/MiBici/datos_tratados_2024_0{}.csv'.format(tiempo), index_col=0))
    
datos_df=pd.concat(year2024)

#Convertir las columnas a datetime
datos_df['inicio_del_viaje'] = pd.to_datetime(datos_df['inicio_del_viaje'])
datos_df['fin_del_viaje'] = pd.to_datetime(datos_df['fin_del_viaje'])
datos_df['tiempo_total'] = pd.to_timedelta(datos_df['tiempo_total'])

## Gráficas por día

In [4]:
vars_semana = ['NA', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

### Datos de gráfica horas vs cantidad

In [5]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour).size().reset_index()
data.columns = ['horas', 'cantidad_NA']
data_dia_total = data

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour).size().reset_index()
data.columns = ['horas', 'cantidad_NA']
data_dia_M = data

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour).size().reset_index()
data.columns = ['horas', 'cantidad_NA']
data_dia_F = data

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour).size().reset_index()
    data.columns = ['horas', 'cantidad_{}'.format(dias)]
    data_dia_total = data_dia_total.merge(
            data[['horas', 'cantidad_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    data_dia_total['cantidad_{}'.format(dias)] = data_dia_total['cantidad_{}'.format(dias)].astype(int)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour).size().reset_index()
    data.columns = ['horas', 'cantidad_{}'.format(dias)]
    data_dia_M = data_dia_M.merge(
            data[['horas', 'cantidad_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    data_dia_M['cantidad_{}'.format(dias)] = data_dia_M['cantidad_{}'.format(dias)].astype(int)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour).size().reset_index()
    data.columns = ['horas', 'cantidad_{}'.format(dias)]
    data_dia_F = data_dia_F.merge(
            data[['horas', 'cantidad_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    data_dia_F['cantidad_{}'.format(dias)] = data_dia_F['cantidad_{}'.format(dias)].astype(int)

### Datos de gráfica horas vs duracion

In [6]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour)['tiempo_total'].mean().reset_index()
data.columns = ['horas', 'duracion_NA']
data['duracion_NA'] = (data['duracion_NA'].dt.total_seconds()/60).round(2)
data_dia_total = data_dia_total.merge(
            data[['horas', 'duracion_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour)['tiempo_total'].mean().reset_index()
data.columns = ['horas', 'duracion_NA']
data['duracion_NA'] = (data['duracion_NA'].dt.total_seconds()/60).round(2)
data_dia_M = data_dia_M.merge(
            data[['horas', 'duracion_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour)['tiempo_total'].mean().reset_index()
data.columns = ['horas', 'duracion_NA']
data['duracion_NA'] = (data['duracion_NA'].dt.total_seconds()/60).round(2)
data_dia_F = data_dia_F.merge(
            data[['horas', 'duracion_NA']],
            on=['horas'],
            how='left').fillna(0)

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour)['tiempo_total'].mean().reset_index()
    data.columns = ['horas', 'duracion_{}'.format(dias)]
    data['duracion_{}'.format(dias)] = (data['duracion_{}'.format(dias)].dt.total_seconds()/60).round(2)
    data_dia_total = data_dia_total.merge(
            data[['horas', 'duracion_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour)['tiempo_total'].mean().reset_index()
    data.columns = ['horas', 'duracion_{}'.format(dias)]
    data['duracion_{}'.format(dias)] = (data['duracion_{}'.format(dias)].dt.total_seconds()/60).round(2)
    data_dia_M = data_dia_M.merge(
            data[['horas', 'duracion_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour)['tiempo_total'].mean().reset_index()
    data.columns = ['horas', 'duracion_{}'.format(dias)]
    data['duracion_{}'.format(dias)] = (data['duracion_{}'.format(dias)].dt.total_seconds()/60).round(2)
    data_dia_F = data_dia_F.merge(
            data[['horas', 'duracion_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)

### Datos de gráfica horas vs edad

In [7]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour)['edad'].mean().reset_index()
data.columns = ['horas', 'edad_NA']
data['edad_NA'] = data['edad_NA'].round(2)
data_dia_total = data_dia_total.merge(
            data[['horas', 'edad_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour)['edad'].mean().reset_index()
data.columns = ['horas', 'edad_NA']
data['edad_NA'] = data['edad_NA'].round(2)
data_dia_M = data_dia_M.merge(
            data[['horas', 'edad_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour)['edad'].mean().reset_index()
data.columns = ['horas', 'edad_NA']
data['edad_NA'] = data['edad_NA'].round(2)
data_dia_F = data_dia_F.merge(
            data[['horas', 'edad_NA']],
            on=['horas'],
            how='left').fillna(0)

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour)['edad'].mean().reset_index()
    data.columns = ['horas', 'edad_{}'.format(dias)]
    data['edad_{}'.format(dias)] = data['edad_{}'.format(dias)].round(2)
    data_dia_total = data_dia_total.merge(
            data[['horas', 'edad_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour)['edad'].mean().reset_index()
    data.columns = ['horas', 'edad_{}'.format(dias)]
    data['edad_{}'.format(dias)] = data['edad_{}'.format(dias)].round(2)
    data_dia_M = data_dia_M.merge(
            data[['horas', 'edad_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour)['edad'].mean().reset_index()
    data.columns = ['horas', 'edad_{}'.format(dias)]
    data['edad_{}'.format(dias)] = data['edad_{}'.format(dias)].round(2)
    data_dia_F = data_dia_F.merge(
            data[['horas', 'edad_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)

### Datos de gráfica horas vs distancia de ruta

In [8]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour)['googlemaps_dis'].mean().reset_index()
data.columns = ['horas', 'maproute_NA']
data['maproute_NA'] = data['maproute_NA'].round(2)
data_dia_total = data_dia_total.merge(
            data[['horas', 'maproute_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour)['googlemaps_dis'].mean().reset_index()
data.columns = ['horas', 'maproute_NA']
data['maproute_NA'] = data['maproute_NA'].round(2)
data_dia_M = data_dia_M.merge(
            data[['horas', 'maproute_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour)['googlemaps_dis'].mean().reset_index()
data.columns = ['horas', 'maproute_NA']
data['maproute_NA'] = data['maproute_NA'].round(2)
data_dia_F = data_dia_F.merge(
            data[['horas', 'maproute_NA']],
            on=['horas'],
            how='left').fillna(0)

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour)['googlemaps_dis'].mean().reset_index()
    data.columns = ['horas', 'maproute_{}'.format(dias)]
    data['maproute_{}'.format(dias)] = data['maproute_{}'.format(dias)].round(2)
    data_dia_total = data_dia_total.merge(
            data[['horas', 'maproute_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour)['googlemaps_dis'].mean().reset_index()
    data.columns = ['horas', 'maproute_{}'.format(dias)]
    data['maproute_{}'.format(dias)] = data['maproute_{}'.format(dias)].round(2)
    data_dia_M = data_dia_M.merge(
            data[['horas', 'maproute_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour)['googlemaps_dis'].mean().reset_index()
    data.columns = ['horas', 'maproute_{}'.format(dias)]
    data['maproute_{}'.format(dias)] = data['maproute_{}'.format(dias)].round(2)
    data_dia_F = data_dia_F.merge(
            data[['horas', 'maproute_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)

### Datos de gráfica horas vs distancia geodesica

In [9]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour)['geodesic_dis'].mean().reset_index()
data.columns = ['horas', 'mapdis_NA']
data['mapdis_NA'] = data['mapdis_NA'].round(2)
data_dia_total = data_dia_total.merge(
            data[['horas', 'mapdis_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour)['geodesic_dis'].mean().reset_index()
data.columns = ['horas', 'mapdis_NA']
data['mapdis_NA'] = data['mapdis_NA'].round(2)
data_dia_M = data_dia_M.merge(
            data[['horas', 'mapdis_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour)['geodesic_dis'].mean().reset_index()
data.columns = ['horas', 'mapdis_NA']
data['mapdis_NA'] = data['mapdis_NA'].round(2)
data_dia_F = data_dia_F.merge(
            data[['horas', 'mapdis_NA']],
            on=['horas'],
            how='left').fillna(0)

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour)['geodesic_dis'].mean().reset_index()
    data.columns = ['horas', 'mapdis_{}'.format(dias)]
    data['mapdis_{}'.format(dias)] = data['mapdis_{}'.format(dias)].round(2)
    data_dia_total = data_dia_total.merge(
            data[['horas', 'mapdis_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour)['geodesic_dis'].mean().reset_index()
    data.columns = ['horas', 'mapdis_{}'.format(dias)]
    data['mapdis_{}'.format(dias)] = data['mapdis_{}'.format(dias)].round(2)
    data_dia_M = data_dia_M.merge(
            data[['horas', 'mapdis_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour)['geodesic_dis'].mean().reset_index()
    data.columns = ['horas', 'mapdis_{}'.format(dias)]
    data['mapdis_{}'.format(dias)] = data['mapdis_{}'.format(dias)].round(2)
    data_dia_F = data_dia_F.merge(
            data[['horas', 'mapdis_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)

### Datos de gráfica horas vs velocidad en ruta

In [10]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_maps'].mean().reset_index()
data.columns = ['horas', 'velroute_NA']
data['velroute_NA'] = data['velroute_NA'].round(2)
data_dia_total = data_dia_total.merge(
            data[['horas', 'velroute_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_maps'].mean().reset_index()
data.columns = ['horas', 'velroute_NA']
data['velroute_NA'] = data['velroute_NA'].round(2)
data_dia_M = data_dia_M.merge(
            data[['horas', 'velroute_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_maps'].mean().reset_index()
data.columns = ['horas', 'velroute_NA']
data['velroute_NA'] = data['velroute_NA'].round(2)
data_dia_F = data_dia_F.merge(
            data[['horas', 'velroute_NA']],
            on=['horas'],
            how='left').fillna(0)

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_maps'].mean().reset_index()
    data.columns = ['horas', 'velroute_{}'.format(dias)]
    data['velroute_{}'.format(dias)] = data['velroute_{}'.format(dias)].round(2)
    data_dia_total = data_dia_total.merge(
            data[['horas', 'velroute_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_maps'].mean().reset_index()
    data.columns = ['horas', 'velroute_{}'.format(dias)]
    data['velroute_{}'.format(dias)] = data['velroute_{}'.format(dias)].round(2)
    data_dia_M = data_dia_M.merge(
            data[['horas', 'velroute_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_maps'].mean().reset_index()
    data.columns = ['horas', 'velroute_{}'.format(dias)]
    data['velroute_{}'.format(dias)] = data['velroute_{}'.format(dias)].round(2)
    data_dia_F = data_dia_F.merge(
            data[['horas', 'velroute_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)

### Datos de gráfica horas vs velocidad geodesica

In [11]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_geo'].mean().reset_index()
data.columns = ['horas', 'veldis_NA']
data['veldis_NA'] = data['veldis_NA'].round(2)
data_dia_total = data_dia_total.merge(
            data[['horas', 'veldis_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_geo'].mean().reset_index()
data.columns = ['horas', 'veldis_NA']
data['veldis_NA'] = data['veldis_NA'].round(2)
data_dia_M = data_dia_M.merge(
            data[['horas', 'veldis_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_geo'].mean().reset_index()
data.columns = ['horas', 'veldis_NA']
data['veldis_NA'] = data['veldis_NA'].round(2)
data_dia_F = data_dia_F.merge(
            data[['horas', 'veldis_NA']],
            on=['horas'],
            how='left').fillna(0)

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_geo'].mean().reset_index()
    data.columns = ['horas', 'veldis_{}'.format(dias)]
    data['veldis_{}'.format(dias)] = data['veldis_{}'.format(dias)].round(2)
    data_dia_total = data_dia_total.merge(
            data[['horas', 'veldis_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_geo'].mean().reset_index()
    data.columns = ['horas', 'veldis_{}'.format(dias)]
    data['veldis_{}'.format(dias)] = data['veldis_{}'.format(dias)].round(2)
    data_dia_M = data_dia_M.merge(
            data[['horas', 'veldis_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour)['vel_prom_geo'].mean().reset_index()
    data.columns = ['horas', 'veldis_{}'.format(dias)]
    data['veldis_{}'.format(dias)] = data['veldis_{}'.format(dias)].round(2)
    data_dia_F = data_dia_F.merge(
            data[['horas', 'veldis_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)

### Datos de gráfica horas vs tarifa promedio

In [12]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.hour)['tarifa'].mean().reset_index()
data.columns = ['horas', 'tarifa_NA']
data['tarifa_NA'] = data['tarifa_NA'].round(2)
data_dia_total = data_dia_total.merge(
            data[['horas', 'tarifa_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.hour)['tarifa'].mean().reset_index()
data.columns = ['horas', 'tarifa_NA']
data['tarifa_NA'] = data['tarifa_NA'].round(2)
data_dia_M = data_dia_M.merge(
            data[['horas', 'tarifa_NA']],
            on=['horas'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.hour)['tarifa'].mean().reset_index()
data.columns = ['horas', 'tarifa_NA']
data['tarifa_NA'] = data['tarifa_NA'].round(2)
data_dia_F = data_dia_F.merge(
            data[['horas', 'tarifa_NA']],
            on=['horas'],
            how='left').fillna(0)

#Caso dias de la semana
for dias in vars_semana[1:]:
    #Datos totales
    data = datos_df[datos_df['dia_semana']==dias].groupby(datos_df['inicio_del_viaje'].dt.hour)['tarifa'].mean().reset_index()
    data.columns = ['horas', 'tarifa_{}'.format(dias)]
    data['tarifa_{}'.format(dias)] = data['tarifa_{}'.format(dias)].round(2)
    data_dia_total = data_dia_total.merge(
            data[['horas', 'tarifa_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.hour)['tarifa'].mean().reset_index()
    data.columns = ['horas', 'tarifa_{}'.format(dias)]
    data['tarifa_{}'.format(dias)] = data['tarifa_{}'.format(dias)].round(2)
    data_dia_M = data_dia_M.merge(
            data[['horas', 'tarifa_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['dia_semana']==dias) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.hour)['tarifa'].mean().reset_index()
    data.columns = ['horas', 'tarifa_{}'.format(dias)]
    data['tarifa_{}'.format(dias)] = data['tarifa_{}'.format(dias)].round(2)
    data_dia_F = data_dia_F.merge(
            data[['horas', 'tarifa_{}'.format(dias)]],
            on=['horas'],
            how='left').fillna(0)

### Guardado de datos

In [28]:
#Completar datos faltantes
data = pd.DataFrame(np.arange(0, 24), columns=['horas'])
data_dia_total = data.merge(
            data_dia_total,
            on='horas',
            how='left').fillna(0)
data_dia_M = data.merge(
            data_dia_M,
            on='horas',
            how='left').fillna(0)
data_dia_F = data.merge(
            data_dia_F,
            on='horas',
            how='left').fillna(0)

#Guardar los datos para gráficas por día
data_dia_total.to_csv('./Datos/MiBici/data_dia_total.csv')
data_dia_M.to_csv('./Datos/MiBici/data_dia_M.csv')
data_dia_F.to_csv('./Datos/MiBici/data_dia_F.csv')

## Gráficas por semana

In [14]:
#Cantidad
    #Total
data = datos_df.groupby('dia_semana').size().reset_index()
data['dia_semana'] = pd.Categorical(data['dia_semana'], categories=vars_semana[1:], ordered=True)
data = data.sort_values('dia_semana')
data.columns = ['dia_semana', 'cantidad']
data_semana = data.reset_index(drop=True)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana').size().reset_index()
data.columns = ['dia_semana', 'cantidad_M']
data_semana = data_semana.merge(
            data[['dia_semana', 'cantidad_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana').size().reset_index()
data.columns = ['dia_semana', 'cantidad_F']
data_semana = data_semana.merge(
            data[['dia_semana', 'cantidad_F']],
            on=['dia_semana'],
            how='left').fillna(0)

#Duración
    #Total
data = datos_df.groupby('dia_semana')['tiempo_total'].mean().reset_index()
data['tiempo_total']=(data['tiempo_total'].dt.total_seconds()/60).round(2)
data.columns = ['dia_semana', 'duracion']
data_semana = data_semana.merge(
            data[['dia_semana', 'duracion']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana')['tiempo_total'].mean().reset_index()
data['tiempo_total']=(data['tiempo_total'].dt.total_seconds()/60).round(2)
data.columns = ['dia_semana', 'duracion_M']
data_semana = data_semana.merge(
            data[['dia_semana', 'duracion_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana')['tiempo_total'].mean().reset_index()
data['tiempo_total']=(data['tiempo_total'].dt.total_seconds()/60).round(2)
data.columns = ['dia_semana', 'duracion_F']
data_semana = data_semana.merge(
            data[['dia_semana', 'duracion_F']],
            on=['dia_semana'],
            how='left').fillna(0)

#Edad
    #Total
data = datos_df.groupby('dia_semana')['edad'].mean().reset_index()
data.columns = ['dia_semana', 'edad']
data['edad'] = data['edad'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'edad']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana')['edad'].mean().reset_index()
data.columns = ['dia_semana', 'edad_M']
data['edad_M'] = data['edad_M'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'edad_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana')['edad'].mean().reset_index()
data.columns = ['dia_semana', 'edad_F']
data['edad_F'] = data['edad_F'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'edad_F']],
            on=['dia_semana'],
            how='left').fillna(0)

#Distancia de ruta
    #Total
data = datos_df.groupby('dia_semana')['googlemaps_dis'].mean().reset_index()
data.columns = ['dia_semana', 'maproute']
data['maproute'] = data['maproute'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'maproute']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana')['googlemaps_dis'].mean().reset_index()
data.columns = ['dia_semana', 'maproute_M']
data['maproute_M'] = data['maproute_M'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'maproute_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana')['googlemaps_dis'].mean().reset_index()
data.columns = ['dia_semana', 'maproute_F']
data['maproute_F'] = data['maproute_F'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'maproute_F']],
            on=['dia_semana'],
            how='left').fillna(0)

#Distancia geodesica
    #Total
data = datos_df.groupby('dia_semana')['geodesic_dis'].mean().reset_index()
data.columns = ['dia_semana', 'mapdis']
data['mapdis'] = data['mapdis'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'mapdis']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana')['geodesic_dis'].mean().reset_index()
data.columns = ['dia_semana', 'mapdis_M']
data['mapdis_M'] = data['mapdis_M'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'mapdis_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana')['geodesic_dis'].mean().reset_index()
data.columns = ['dia_semana', 'mapdis_F']
data['mapdis_F'] = data['mapdis_F'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'mapdis_F']],
            on=['dia_semana'],
            how='left').fillna(0)

#Velocidad en ruta
    #Total
data = datos_df.groupby('dia_semana')['vel_prom_maps'].mean().reset_index()
data.columns = ['dia_semana', 'velroute']
data['velroute'] = data['velroute'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'velroute']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana')['vel_prom_maps'].mean().reset_index()
data.columns = ['dia_semana', 'velroute_M']
data['velroute_M'] = data['velroute_M'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'velroute_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana')['vel_prom_maps'].mean().reset_index()
data.columns = ['dia_semana', 'velroute_F']
data['velroute_F'] = data['velroute_F'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'velroute_F']],
            on=['dia_semana'],
            how='left').fillna(0)

#Velocidad geodesica
    #Total
data = datos_df.groupby('dia_semana')['vel_prom_geo'].mean().reset_index()
data.columns = ['dia_semana', 'veldis']
data['veldis'] = data['veldis'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'veldis']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana')['vel_prom_geo'].mean().reset_index()
data.columns = ['dia_semana', 'veldis_M']
data['veldis_M'] = data['veldis_M'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'veldis_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana')['vel_prom_geo'].mean().reset_index()
data.columns = ['dia_semana', 'veldis_F']
data['veldis_F'] = data['veldis_F'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'veldis_F']],
            on=['dia_semana'],
            how='left').fillna(0)

#Tarifa promedio
    #Total
data = datos_df.groupby('dia_semana')['tarifa'].mean().reset_index()
data.columns = ['dia_semana', 'tarifa']
data['tarifa'] = data['tarifa'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'tarifa']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby('dia_semana')['tarifa'].mean().reset_index()
data.columns = ['dia_semana', 'tarifa_M']
data['tarifa_M'] = data['tarifa_M'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'tarifa_M']],
            on=['dia_semana'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby('dia_semana')['tarifa'].mean().reset_index()
data.columns = ['dia_semana', 'tarifa_F']
data['tarifa_F'] = data['tarifa_F'].round(2)
data_semana = data_semana.merge(
            data[['dia_semana', 'tarifa_F']],
            on=['dia_semana'],
            how='left').fillna(0)

### Guardado de datos

In [15]:
#Guardar los datos para gráficas por semana
data_semana.to_csv('./Datos/MiBici/data_semana.csv')

## Gráficas por mes

In [16]:
vars_mes = ['NA', 'January','February','March','April','May','June','July','August']

### Datos de gráfica dias vs cantidad

In [17]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day).size().reset_index()
data.columns = ['dias', 'cantidad_NA']
data_mes_total = data

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day).size().reset_index()
data.columns = ['dias', 'cantidad_NA']
data_mes_M = data

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day).size().reset_index()
data.columns = ['dias', 'cantidad_NA']
data_mes_F = data

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day).size().reset_index()
    data.columns = ['dias', 'cantidad_{}'.format(meses)]
    data_mes_total = data_mes_total.merge(
            data[['dias', 'cantidad_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    data_mes_total['cantidad_{}'.format(meses)] = data_mes_total['cantidad_{}'.format(meses)].astype(int)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day).size().reset_index()
    data.columns = ['dias', 'cantidad_{}'.format(meses)]
    data_mes_M = data_mes_M.merge(
            data[['dias', 'cantidad_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    data_mes_M['cantidad_{}'.format(meses)] = data_mes_M['cantidad_{}'.format(meses)].astype(int)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day).size().reset_index()
    data.columns = ['dias', 'cantidad_{}'.format(meses)]
    data_mes_F = data_mes_F.merge(
            data[['dias', 'cantidad_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    data_mes_F['cantidad_{}'.format(meses)] = data_mes_F['cantidad_{}'.format(meses)].astype(int)

### Datos de gráfica dias vs duracion

In [18]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day)['tiempo_total'].mean().reset_index()
data.columns = ['dias', 'duracion_NA']
data['duracion_NA'] = (data['duracion_NA'].dt.total_seconds()/60).round(2)
data_mes_total = data_mes_total.merge(
            data[['dias', 'duracion_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day)['tiempo_total'].mean().reset_index()
data.columns = ['dias', 'duracion_NA']
data['duracion_NA'] = (data['duracion_NA'].dt.total_seconds()/60).round(2)
data_mes_M = data_mes_M.merge(
            data[['dias', 'duracion_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day)['tiempo_total'].mean().reset_index()
data.columns = ['dias', 'duracion_NA']
data['duracion_NA'] = (data['duracion_NA'].dt.total_seconds()/60).round(2)
data_mes_F = data_mes_F.merge(
            data[['dias', 'duracion_NA']],
            on=['dias'],
            how='left').fillna(0)

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day)['tiempo_total'].mean().reset_index()
    data.columns = ['dias', 'duracion_{}'.format(meses)]
    data['duracion_{}'.format(meses)] = (data['duracion_{}'.format(meses)].dt.total_seconds()/60).round(2)
    data_mes_total = data_mes_total.merge(
            data[['dias', 'duracion_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day)['tiempo_total'].mean().reset_index()
    data.columns = ['dias', 'duracion_{}'.format(meses)]
    data['duracion_{}'.format(meses)] = (data['duracion_{}'.format(meses)].dt.total_seconds()/60).round(2)
    data_mes_M = data_mes_M.merge(
            data[['dias', 'duracion_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day)['tiempo_total'].mean().reset_index()
    data.columns = ['dias', 'duracion_{}'.format(meses)]
    data['duracion_{}'.format(meses)] = (data['duracion_{}'.format(meses)].dt.total_seconds()/60).round(2)
    data_mes_F = data_mes_F.merge(
            data[['dias', 'duracion_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)

### Datos de gráfica dias vs edad

In [19]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day)['edad'].mean().reset_index()
data.columns = ['dias', 'edad_NA']
data['edad_NA'] = data['edad_NA'].round(2)
data_mes_total = data_mes_total.merge(
            data[['dias', 'edad_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day)['edad'].mean().reset_index()
data.columns = ['dias', 'edad_NA']
data['edad_NA'] = data['edad_NA'].round(2)
data_mes_M = data_mes_M.merge(
            data[['dias', 'edad_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day)['edad'].mean().reset_index()
data.columns = ['dias', 'edad_NA']
data['edad_NA'] = data['edad_NA'].round(2)
data_mes_F = data_mes_F.merge(
            data[['dias', 'edad_NA']],
            on=['dias'],
            how='left').fillna(0)

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day)['edad'].mean().reset_index()
    data.columns = ['dias', 'edad_{}'.format(meses)]
    data['edad_{}'.format(meses)] = data['edad_{}'.format(meses)].round(2)
    data_mes_total = data_mes_total.merge(
            data[['dias', 'edad_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day)['edad'].mean().reset_index()
    data.columns = ['dias', 'edad_{}'.format(meses)]
    data['edad_{}'.format(meses)] = data['edad_{}'.format(meses)].round(2)
    data_mes_M = data_mes_M.merge(
            data[['dias', 'edad_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day)['edad'].mean().reset_index()
    data.columns = ['dias', 'edad_{}'.format(meses)]
    data['edad_{}'.format(meses)] = data['edad_{}'.format(meses)].round(2)
    data_mes_F = data_mes_F.merge(
            data[['dias', 'edad_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)

### Datos de gráfica dias vs distancia de ruta

In [20]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day)['googlemaps_dis'].mean().reset_index()
data.columns = ['dias', 'maproute_NA']
data['maproute_NA'] = data['maproute_NA'].round(2)
data_mes_total = data_mes_total.merge(
            data[['dias', 'maproute_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day)['googlemaps_dis'].mean().reset_index()
data.columns = ['dias', 'maproute_NA']
data['maproute_NA'] = data['maproute_NA'].round(2)
data_mes_M = data_mes_M.merge(
            data[['dias', 'maproute_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day)['googlemaps_dis'].mean().reset_index()
data.columns = ['dias', 'maproute_NA']
data['maproute_NA'] = data['maproute_NA'].round(2)
data_mes_F = data_mes_F.merge(
            data[['dias', 'maproute_NA']],
            on=['dias'],
            how='left').fillna(0)

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day)['googlemaps_dis'].mean().reset_index()
    data.columns = ['dias', 'maproute_{}'.format(meses)]
    data['maproute_{}'.format(meses)] = data['maproute_{}'.format(meses)].round(2)
    data_mes_total = data_mes_total.merge(
            data[['dias', 'maproute_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day)['googlemaps_dis'].mean().reset_index()
    data.columns = ['dias', 'maproute_{}'.format(meses)]
    data['maproute_{}'.format(meses)] = data['maproute_{}'.format(meses)].round(2)
    data_mes_M = data_mes_M.merge(
            data[['dias', 'maproute_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day)['googlemaps_dis'].mean().reset_index()
    data.columns = ['dias', 'maproute_{}'.format(meses)]
    data['maproute_{}'.format(meses)] = data['maproute_{}'.format(meses)].round(2)
    data_mes_F = data_mes_F.merge(
            data[['dias', 'maproute_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)

### Datos gráfica dias vs distancia geodesica

In [21]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day)['geodesic_dis'].mean().reset_index()
data.columns = ['dias', 'mapdis_NA']
data['mapdis_NA'] = data['mapdis_NA'].round(2)
data_mes_total = data_mes_total.merge(
            data[['dias', 'mapdis_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day)['geodesic_dis'].mean().reset_index()
data.columns = ['dias', 'mapdis_NA']
data['mapdis_NA'] = data['mapdis_NA'].round(2)
data_mes_M = data_mes_M.merge(
            data[['dias', 'mapdis_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day)['geodesic_dis'].mean().reset_index()
data.columns = ['dias', 'mapdis_NA']
data['mapdis_NA'] = data['mapdis_NA'].round(2)
data_mes_F = data_mes_F.merge(
            data[['dias', 'mapdis_NA']],
            on=['dias'],
            how='left').fillna(0)

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day)['geodesic_dis'].mean().reset_index()
    data.columns = ['dias', 'mapdis_{}'.format(meses)]
    data['mapdis_{}'.format(meses)] = data['mapdis_{}'.format(meses)].round(2)
    data_mes_total = data_mes_total.merge(
            data[['dias', 'mapdis_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day)['geodesic_dis'].mean().reset_index()
    data.columns = ['dias', 'mapdis_{}'.format(meses)]
    data['mapdis_{}'.format(meses)] = data['mapdis_{}'.format(meses)].round(2)
    data_mes_M = data_mes_M.merge(
            data[['dias', 'mapdis_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day)['geodesic_dis'].mean().reset_index()
    data.columns = ['dias', 'mapdis_{}'.format(meses)]
    data['mapdis_{}'.format(meses)] = data['mapdis_{}'.format(meses)].round(2)
    data_mes_F = data_mes_F.merge(
            data[['dias', 'mapdis_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)

### Datos gráfica dias vs velocidad en ruta

In [22]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_maps'].mean().reset_index()
data.columns = ['dias', 'velroute_NA']
data['velroute_NA'] = data['velroute_NA'].round(2)
data_mes_total = data_mes_total.merge(
            data[['dias', 'velroute_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_maps'].mean().reset_index()
data.columns = ['dias', 'velroute_NA']
data['velroute_NA'] = data['velroute_NA'].round(2)
data_mes_M = data_mes_M.merge(
            data[['dias', 'velroute_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_maps'].mean().reset_index()
data.columns = ['dias', 'velroute_NA']
data['velroute_NA'] = data['velroute_NA'].round(2)
data_mes_F = data_mes_F.merge(
            data[['dias', 'velroute_NA']],
            on=['dias'],
            how='left').fillna(0)

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_maps'].mean().reset_index()
    data.columns = ['dias', 'velroute_{}'.format(meses)]
    data['velroute_{}'.format(meses)] = data['velroute_{}'.format(meses)].round(2)
    data_mes_total = data_mes_total.merge(
            data[['dias', 'velroute_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_maps'].mean().reset_index()
    data.columns = ['dias', 'velroute_{}'.format(meses)]
    data['velroute_{}'.format(meses)] = data['velroute_{}'.format(meses)].round(2)
    data_mes_M = data_mes_M.merge(
            data[['dias', 'velroute_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_maps'].mean().reset_index()
    data.columns = ['dias', 'velroute_{}'.format(meses)]
    data['velroute_{}'.format(meses)] = data['velroute_{}'.format(meses)].round(2)
    data_mes_F = data_mes_F.merge(
            data[['dias', 'velroute_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)

### Datos de gráfico dias vs velocidad geodesica

In [23]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_geo'].mean().reset_index()
data.columns = ['dias', 'veldis_NA']
data['veldis_NA'] = data['veldis_NA'].round(2)
data_mes_total = data_mes_total.merge(
            data[['dias', 'veldis_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_geo'].mean().reset_index()
data.columns = ['dias', 'veldis_NA']
data['veldis_NA'] = data['veldis_NA'].round(2)
data_mes_M = data_mes_M.merge(
            data[['dias', 'veldis_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_geo'].mean().reset_index()
data.columns = ['dias', 'veldis_NA']
data['veldis_NA'] = data['veldis_NA'].round(2)
data_mes_F = data_mes_F.merge(
            data[['dias', 'veldis_NA']],
            on=['dias'],
            how='left').fillna(0)

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_geo'].mean().reset_index()
    data.columns = ['dias', 'veldis_{}'.format(meses)]
    data['veldis_{}'.format(meses)] = data['veldis_{}'.format(meses)].round(2)
    data_mes_total = data_mes_total.merge(
            data[['dias', 'veldis_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_geo'].mean().reset_index()
    data.columns = ['dias', 'veldis_{}'.format(meses)]
    data['veldis_{}'.format(meses)] = data['veldis_{}'.format(meses)].round(2)
    data_mes_M = data_mes_M.merge(
            data[['dias', 'veldis_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day)['vel_prom_geo'].mean().reset_index()
    data.columns = ['dias', 'veldis_{}'.format(meses)]
    data['veldis_{}'.format(meses)] = data['veldis_{}'.format(meses)].round(2)
    data_mes_F = data_mes_F.merge(
            data[['dias', 'veldis_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)

### Datos gráfica dias vs tarifa promedio

In [24]:
#Caso NA
    #Datos totales
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.day)['tarifa'].mean().reset_index()
data.columns = ['dias', 'tarifa_NA']
data['tarifa_NA'] = data['tarifa_NA'].round(2)
data_mes_total = data_mes_total.merge(
            data[['dias', 'tarifa_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.day)['tarifa'].mean().reset_index()
data.columns = ['dias', 'tarifa_NA']
data['tarifa_NA'] = data['tarifa_NA'].round(2)
data_mes_M = data_mes_M.merge(
            data[['dias', 'tarifa_NA']],
            on=['dias'],
            how='left').fillna(0)

    #Datos Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.day)['tarifa'].mean().reset_index()
data.columns = ['dias', 'tarifa_NA']
data['tarifa_NA'] = data['tarifa_NA'].round(2)
data_mes_F = data_mes_F.merge(
            data[['dias', 'tarifa_NA']],
            on=['dias'],
            how='left').fillna(0)

#Caso dias de la semana
for meses in vars_mes[1:]:
    #Datos totales
    data = datos_df[datos_df['inicio_del_viaje'].dt.month_name()==meses].groupby(datos_df['inicio_del_viaje'].dt.day)['tarifa'].mean().reset_index()
    data.columns = ['dias', 'tarifa_{}'.format(meses)]
    data['tarifa_{}'.format(meses)] = data['tarifa_{}'.format(meses)].round(2)
    data_mes_total = data_mes_total.merge(
            data[['dias', 'tarifa_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Hombres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='M')].groupby(datos_df['inicio_del_viaje'].dt.day)['tarifa'].mean().reset_index()
    data.columns = ['dias', 'tarifa_{}'.format(meses)]
    data['tarifa_{}'.format(meses)] = data['tarifa_{}'.format(meses)].round(2)
    data_mes_M = data_mes_M.merge(
            data[['dias', 'tarifa_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)
    
    #Datos Mujeres
    data = datos_df[(datos_df['inicio_del_viaje'].dt.month_name()==meses) & (datos_df['genero']=='F')].groupby(datos_df['inicio_del_viaje'].dt.day)['tarifa'].mean().reset_index()
    data.columns = ['dias', 'tarifa_{}'.format(meses)]
    data['tarifa_{}'.format(meses)] = data['tarifa_{}'.format(meses)].round(2)
    data_mes_F = data_mes_F.merge(
            data[['dias', 'tarifa_{}'.format(meses)]],
            on=['dias'],
            how='left').fillna(0)

### Guardado de datos

In [25]:
#Guardar los datos para gráficas por mes
data_mes_total.to_csv('./Datos/MiBici/data_mes_total.csv')
data_mes_M.to_csv('./Datos/MiBici/data_mes_M.csv')
data_mes_F.to_csv('./Datos/MiBici/data_mes_F.csv')

## Gráficas por año

In [26]:
#Cantidad
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name()).size().reset_index()
data['inicio_del_viaje'] = pd.Categorical(data['inicio_del_viaje'], categories=vars_mes[1:], ordered=True)
data = data.sort_values('inicio_del_viaje')
data.columns = ['mes', 'cantidad']
data_año = data.reset_index(drop=True)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name()).size().reset_index()
data.columns = ['mes', 'cantidad_M']
data_año = data_año.merge(
            data[['mes', 'cantidad_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name()).size().reset_index()
data.columns = ['mes', 'cantidad_F']
data_año = data_año.merge(
            data[['mes', 'cantidad_F']],
            on=['mes'],
            how='left').fillna(0)

#Duración
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name())['tiempo_total'].mean().reset_index()
data['tiempo_total']=(data['tiempo_total'].dt.total_seconds()/60).round(2)
data.columns = ['mes', 'duracion']
data_año = data_año.merge(
            data[['mes', 'duracion']],
            on=['mes'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['tiempo_total'].mean().reset_index()
data['tiempo_total']=(data['tiempo_total'].dt.total_seconds()/60).round(2)
data.columns = ['mes', 'duracion_M']
data_año = data_año.merge(
            data[['mes', 'duracion_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['tiempo_total'].mean().reset_index()
data['tiempo_total']=(data['tiempo_total'].dt.total_seconds()/60).round(2)
data.columns = ['mes', 'duracion_F']
data_año = data_año.merge(
            data[['mes', 'duracion_F']],
            on=['mes'],
            how='left').fillna(0)

#Edad
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name())['edad'].mean().reset_index()
data.columns = ['mes', 'edad']
data['edad'] = data['edad'].round(2)
data_año = data_año.merge(
            data[['mes', 'edad']],
            on=['mes'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['edad'].mean().reset_index()
data.columns = ['mes', 'edad_M']
data['edad_M'] = data['edad_M'].round(2)
data_año = data_año.merge(
            data[['mes', 'edad_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['edad'].mean().reset_index()
data.columns = ['mes', 'edad_F']
data['edad_F'] = data['edad_F'].round(2)
data_año = data_año.merge(
            data[['mes', 'edad_F']],
            on=['mes'],
            how='left').fillna(0)

#Distancia de ruta
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name())['googlemaps_dis'].mean().reset_index()
data.columns = ['mes', 'maproute']
data['maproute'] = data['maproute'].round(2)
data_año = data_año.merge(
            data[['mes', 'maproute']],
            on=['mes'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['googlemaps_dis'].mean().reset_index()
data.columns = ['mes', 'maproute_M']
data['maproute_M'] = data['maproute_M'].round(2)
data_año = data_año.merge(
            data[['mes', 'maproute_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['googlemaps_dis'].mean().reset_index()
data.columns = ['mes', 'maproute_F']
data['maproute_F'] = data['maproute_F'].round(2)
data_año = data_año.merge(
            data[['mes', 'maproute_F']],
            on=['mes'],
            how='left').fillna(0)

#Distancia geodesica
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name())['geodesic_dis'].mean().reset_index()
data.columns = ['mes', 'mapdis']
data['mapdis'] = data['mapdis'].round(2)
data_año = data_año.merge(
            data[['mes', 'mapdis']],
            on=['mes'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['geodesic_dis'].mean().reset_index()
data.columns = ['mes', 'mapdis_M']
data['mapdis_M'] = data['mapdis_M'].round(2)
data_año = data_año.merge(
            data[['mes', 'mapdis_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['geodesic_dis'].mean().reset_index()
data.columns = ['mes', 'mapdis_F']
data['mapdis_F'] = data['mapdis_F'].round(2)
data_año = data_año.merge(
            data[['mes', 'mapdis_F']],
            on=['mes'],
            how='left').fillna(0)

#Velocidad en ruta
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name())['vel_prom_maps'].mean().reset_index()
data.columns = ['mes', 'velroute']
data['velroute'] = data['velroute'].round(2)
data_año = data_año.merge(
            data[['mes', 'velroute']],
            on=['mes'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['vel_prom_maps'].mean().reset_index()
data.columns = ['mes', 'velroute_M']
data['velroute_M'] = data['velroute_M'].round(2)
data_año = data_año.merge(
            data[['mes', 'velroute_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['vel_prom_maps'].mean().reset_index()
data.columns = ['mes', 'velroute_F']
data['velroute_F'] = data['velroute_F'].round(2)
data_año = data_año.merge(
            data[['mes', 'velroute_F']],
            on=['mes'],
            how='left').fillna(0)

#Velocidad geodesica
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name())['vel_prom_geo'].mean().reset_index()
data.columns = ['mes', 'veldis']
data['veldis'] = data['veldis'].round(2)
data_año = data_año.merge(
            data[['mes', 'veldis']],
            on=['mes'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['vel_prom_geo'].mean().reset_index()
data.columns = ['mes', 'veldis_M']
data['veldis_M'] = data['veldis_M'].round(2)
data_año = data_año.merge(
            data[['mes', 'veldis_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['vel_prom_geo'].mean().reset_index()
data.columns = ['mes', 'veldis_F']
data['veldis_F'] = data['veldis_F'].round(2)
data_año = data_año.merge(
            data[['mes', 'veldis_F']],
            on=['mes'],
            how='left').fillna(0)

#Tarifa promedio
    #Total
data = datos_df.groupby(datos_df['inicio_del_viaje'].dt.month_name())['tarifa'].mean().reset_index()
data.columns = ['mes', 'tarifa']
data['tarifa'] = data['tarifa'].round(2)
data_año = data_año.merge(
            data[['mes', 'tarifa']],
            on=['mes'],
            how='left').fillna(0)

    #Hombres
data = datos_df[datos_df['genero']=='M'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['tarifa'].mean().reset_index()
data.columns = ['mes', 'tarifa_M']
data['tarifa_M'] = data['tarifa_M'].round(2)
data_año = data_año.merge(
            data[['mes', 'tarifa_M']],
            on=['mes'],
            how='left').fillna(0)

    #Mujeres
data = datos_df[datos_df['genero']=='F'].groupby(datos_df['inicio_del_viaje'].dt.month_name())['tarifa'].mean().reset_index()
data.columns = ['mes', 'tarifa_F']
data['tarifa_F'] = data['tarifa_F'].round(2)
data_año = data_año.merge(
            data[['mes', 'tarifa_F']],
            on=['mes'],
            how='left').fillna(0)

### Guardado de datos

In [27]:
#Guardar los datos para gráficas por año
data_año.to_csv('./Datos/MiBici/data_año.csv')

In [39]:
#Estaciones de origen
top_stations = datos_df['origen_id'].value_counts().nlargest(10).index
data_grouped = datos_df[datos_df['origen_id'].isin(top_stations)].groupby(
    ['origen_id', pd.Grouper(key='inicio_del_viaje', freq='M')]
).size().reset_index(name='cantidad_viajes')

data_grouped.to_csv('./Datos/MiBici/estaciones_origen.csv')

#Estaciones de destino
top_stations = datos_df['destino_id'].value_counts().nlargest(10).index
data_grouped = datos_df[datos_df['destino_id'].isin(top_stations)].groupby(
    ['destino_id', pd.Grouper(key='inicio_del_viaje', freq='M')]
).size().reset_index(name='cantidad_viajes')

data_grouped.to_csv('./Datos/MiBici/estaciones_destino.csv')

In [65]:
datos_correlacion = datos_df[['edad', 'tiempo_total']].copy()
datos_correlacion['tiempo_total'] = datos_correlacion['tiempo_total'].dt.total_seconds().astype(int)
datos_correlacion = datos_correlacion.sample(frac=0.1)
datos_correlacion.to_csv('./Datos/MiBici/correlacion_1.csv')

## Clusterización

In [75]:
#Leer y concatenar dataframes
year2024=[]
year2024.append(pd.read_csv('./Datos/MiBici/datos_tratados_2024_01.csv', index_col=0))
    
datos_df=pd.concat(year2024)

#Convertir las columnas a datetime
datos_df['inicio_del_viaje'] = pd.to_datetime(datos_df['inicio_del_viaje'])
datos_df['fin_del_viaje'] = pd.to_datetime(datos_df['fin_del_viaje'])
datos_df['tiempo_total'] = pd.to_timedelta(datos_df['tiempo_total'])

In [77]:
#Pasar columnas importantes y modificarlas
cluster_df = datos_df[['genero', 'edad', 'tiempo_total', 'dia_semana', 'googlemaps_dis', 'vel_prom_maps']].reset_index().drop('viaje_id', axis=1)
cluster_df['tiempo_total'] = cluster_df['tiempo_total'].dt.total_seconds().astype(int)

In [79]:
#Normalizar columnas numericas
columnas_numericas = ['edad', 'tiempo_total','googlemaps_dis', 'vel_prom_maps']
cluster_df[columnas_numericas] = StandardScaler().fit_transform(cluster_df[columnas_numericas])

In [81]:
#Normalizar datos categoricos
cluster_df['dia_semana'] = cluster_df['dia_semana'].map({'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6})

In [83]:
#Guardar dataframe
cluster_df.to_csv('./Datos/MiBici/cluster_1.csv')